In [7]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import gc
import os
import time
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import nltk
import nltk.corpus
import nltk.tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
import gensim.downloader
import string
import gensim
from string import punctuation
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from collections import Counter
from torch.utils.data.sampler import WeightedRandomSampler
from torchsampler import ImbalancedDatasetSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from gensim.models import KeyedVectors

In [28]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [29]:
def reshaping(lst):
    ans = []
    for i in range(len(lst)):
        ans.append([lst[i]])
    return ans

# LR

In [31]:
count_iter = 0
times = []
times.append(time.time())
penalty = ['l1', 'l2', 'elasticnet', None]
class_weight = ['balanced',None]

for weight in [None,'balanced']:   
    for C in [0.1]:
        if weight == 'balanced':
            temp_weight = 'balanced'
        else:
            temp_weight = 'None'
        print(temp_weight, C)
        clf = LogisticRegression(class_weight=weight, C=C)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        times.append(time.time())
        test_acc = accuracy_score(y_test, pred)
        test_f1 = f1_score(y_test, pred, average='weighted') 
        print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
        temp_ans = [lemm_column_name, temp_weight, C, test_acc, test_f1,times[-1]-times[-2]]
        df_result.loc[df_result.shape[0]-1] = temp_ans
        count_iter += 1
        clf = None
        del clf
        pred = None
        del pred

balanced 0.1
acc 0.691 f1 0.696 time 3.729391098022461


# NB

In [35]:
df_result = pd.DataFrame([], columns=['lemm_column','test_acc','test_f1', 'time'])
count_iter = 0
times = []
times.append(time.time())
clf = GaussianNB()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
times.append(time.time())
test_acc = accuracy_score(y_test, pred)
test_f1 = f1_score(y_test, pred, average='weighted') 
print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
temp_ans = [lemm_column_name, test_acc, test_f1,times[-1]-times[-2]]
df_result.loc[df_result.shape[0]-1] = temp_ans
df_result.to_excel('./Результаты классический ml/df_nb_{}.xlsx'.format(count_iter),index=False)
count_iter += 1
clf = None
del clf
pred = None
del pred

acc 0.516 f1 0.409 time 4.924217700958252


# dt

In [36]:
for weight in [None,'balanced']:
#     for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    for criterion in ['gini', 'entropy', 'log_loss']:
        if weight == 'balanced':
            temp_weight = 'balanced'
        else:
            temp_weight = 'None'
        print(temp_weight, criterion)
        clf = DecisionTreeClassifier(class_weight=weight, criterion=criterion)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        times.append(time.time())
        test_acc = accuracy_score(y_test, pred)
        test_f1 = f1_score(y_test, pred, average='weighted') 
        print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
        clf = None
        del clf
        pred = None
        del pred

None gini 1
acc 0.548 f1 0.548 time 144.21212124824524
None entropy 1
acc 0.551 f1 0.551 time 189.13862299919128
None log_loss 1
acc 0.55 f1 0.55 time 187.95779705047607
balanced gini 1
acc 0.548 f1 0.547 time 252.28252577781677
balanced entropy 1
acc 0.551 f1 0.55 time 270.14551854133606
balanced log_loss 1
acc 0.552 f1 0.552 time 268.8990321159363


# SVM

In [34]:
for weight in [None,'balanced']:
#     for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    for kernel in ['poly']:
#         for C in [0.01, 0.1, 1]:
        for C in [0.1]:
            if weight == 'balanced':
                temp_weight = 'balanced'
            else:
                temp_weight = 'None'
            print(temp_weight, kernel, C)
            clf = SVC(class_weight=weight, C=C, kernel=kernel)
            clf.fit(X_train, y_train)
            pred = clf.predict(X_test)
            times.append(time.time())
            test_acc = accuracy_score(y_test, pred)
            test_f1 = f1_score(y_test, pred, average='weighted') 
            print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
            clf = None
            del clf
            pred = None
            del pred

balanced poly 0.1
acc 0.564 f1 0.503 time 96.32277393341064


ValueError: cannot set a row with mismatched columns

# KNN

In [32]:
for weight in ['uniform']:    
    for neigh in [9]:
        print(weight, neigh)
        clf = KNeighborsClassifier(n_neighbors=neigh)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        times.append(time.time())
        test_acc = accuracy_score(y_test, pred)
        test_f1 = f1_score(y_test, pred, average='weighted') 
        print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
        clf = None
        del clf
        pred = None
        del pred

uniform 9
acc 0.713 f1 0.696 time 14.179454803466797


# LDA

In [23]:
# for weight in [None,'balanced']:
for shrinkage in ['auto', None]:    
    for solver in ['svd', 'lsqr', 'eigen']:
        if shrinkage == 'auto':
            temp_shrinkage = 'auto'
        else:
            temp_shrinkage = 'None'
        try:
            print(temp_shrinkage, solver)
            clf = LinearDiscriminantAnalysis(shrinkage=shrinkage, solver=solver)
            clf.fit(X_train, y_train)
            pred = clf.predict(X_test)
            times.append(time.time())
            test_acc = accuracy_score(y_test, pred)
            test_f1 = f1_score(y_test, pred, average='weighted') 
            print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
            clf = None
            del clf
            pred = None
            del pred
        except:
            print('error')

auto svd
error
auto lsqr
acc 0.629 f1 0.622 time 9.36988377571106
auto eigen
acc 0.629 f1 0.622 time 9.002815008163452
None svd
acc 0.629 f1 0.622 time 10.618637323379517
None lsqr
acc 0.629 f1 0.622 time 6.040789604187012
None eigen
acc 0.629 f1 0.622 time 6.164265394210815


# XGboost

In [25]:
booster = ['gbtree', 'gblinear', 'dart']
eta = [0.1, 0.3, 0.5]
# for weight in [None,'balanced']:
for eta in [0.1, 0.3, 0.5]:    
#     for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    for booster in ['gbtree', 'gblinear', 'dart']:
        print(eta, booster)
        clf = XGBClassifier(device='cuda', eta=eta, booster=booster)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        times.append(time.time())
        test_acc = accuracy_score(y_test, pred)
        test_f1 = f1_score(y_test, pred, average='weighted') 
        print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
        clf = None
        del clf
        pred = None
        del pred

0.1 gbtree
acc 0.675 f1 0.675 time 22.27203869819641
0.1 gblinear
acc 0.633 f1 0.627 time 22.450019598007202
0.1 dart
acc 0.675 f1 0.675 time 31.458714723587036
0.3 gbtree
acc 0.685 f1 0.685 time 22.093270301818848
0.3 gblinear
acc 0.635 f1 0.629 time 22.373468160629272
0.3 dart
acc 0.685 f1 0.685 time 31.531245231628418
0.5 gbtree
acc 0.683 f1 0.683 time 21.534498929977417
0.5 gblinear
acc 0.635 f1 0.628 time 22.452567100524902
0.5 dart
acc 0.683 f1 0.683 time 31.19697070121765


# Catboost

In [33]:
booster = ['gbtree', 'gblinear', 'dart']
eta = [0.1, 0.3, 0.5]
loss_function = ['RMSE','Logloss','MAE','CrossEntropy', 'Quantile','LogLinQuantile', 'Lq','MultiRMSE','MultiClass',
                 'MultiClassOneVsAll','MultiLogloss','MultiCrossEntropy','MAPE','Poisson',
                 'PairLogit','PairLogitPairwise','QueryRMSE','QuerySoftMax', 'Tweedie','YetiRank',
                 'YetiRankPairwise','StochasticFilter','StochasticRank']
learning_rate = [0.01, 0.03, 0.05]

times.append(time.time())
for loss in ['MultiClass']:    
    for learning_rate in [0.05]:
        print(loss, learning_rate)
        clf = CatBoostClassifier(task_type="GPU",
                                devices='0:0', 
                                loss_function=loss, 
                                learning_rate=learning_rate,
                                verbose=0)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        times.append(time.time())
        test_acc = accuracy_score(y_test, pred)
        test_f1 = f1_score(y_test, pred, average='weighted') 
        print('acc {} f1 {} time {}'.format(round(test_acc,3),round(test_f1,3), times[-1]-times[-2]))
        clf = None
        del clf
        pred = None
        del pred

MultiClass 0.05
acc 0.741 f1 0.73 time 21.89696455001831


In [38]:
from joblib import dump
from joblib import load
dump(clf_svm, 'clf_svm.joblib')
dump(clf_cat, 'clf_cat.joblib')
dump(clf_knn, 'clf_knn.joblib')
dump(clf_lr, 'clf_lr.joblib')

['clf_lr.joblib']

In [40]:
clf = load('./Модели ML/clf_lr.joblib')

In [41]:
pred = clf.predict(X_test)